In [25]:
import math
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

# Loading data
data = pd.read_csv('temperature_filtered.csv', header=None)

df = pd.DataFrame()
df["unix"] = data[2]
df["temp"] = data[6]

train_size = int(len(df) * 0.8)

train_df, test_df = df[:train_size], df[train_size+1:]
print(f"train: {train_df.shape}; test: {test_df.shape}")

def create_sequence(input_data: pd.DataFrame, target_column, sequence_length):
    sequences = []
    data_size = len(input_data)

    for i in tqdm(range(data_size - sequence_length)):
        sequence = input_data[i:i+sequence_length]

        label_position = i + sequence_length
        label = input_data.iloc[label_position][target_column]

        sequences.append((sequence, label))

    return sequence

train: (268104, 2); test: (67026, 2)


In [69]:
SEQUENCE_LENGTH = 60
train_sequence = create_sequence(train_df, "temp", SEQUENCE_LENGTH)
test_sequence = create_sequence(test_df, "temp", SEQUENCE_LENGTH)

  0%|          | 0/268044 [00:00<?, ?it/s]

  0%|          | 0/66966 [00:00<?, ?it/s]

In [9]:
pl.seed_everything(42)

Global seed set to 42


42

In [41]:
class TempDataSet(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(
            sequence = torch.Tensor(sequence.to_numpy()),
            label = torch.Tensor(label).float()
        )

In [20]:
class TempPredictionModel(nn.Module):
    def __init__(self, n_features, n_hidden=128, n_layers=2):
        super().__init__()

        self.n_hidden = n_hidden

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            batch_first=True,
            num_layers=n_layers,
            dropout=0.2,)

        self.regressor = nn.Linear(n_hidden, 1)

    def forward(self, x):
        self.lstm.flatten_parameters()

        _, (hidden, _) = self.lstm(x)
        out = hidden[-1]

        return self.regressor(out)

In [66]:
class TempDataModule(pl.LightningDataModule):
    def __init__(
        self, train_sequence, test_sequence, batch_size=8
    ):
        super().__init__()
        self.train_sequence = train_sequence
        self.test_sequence = test_sequence
        self.batch_size = batch_size
    
    def setup(self):
        self.train_dataset = TempDataSet(self.train_sequence)
        self.test_dataset = TempDataSet(self.test_sequence)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            shuffle=False,
            num_workers=1
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            shuffle=False,
            num_workers=1
        )

In [62]:
class TempPredictor(pl.LightningModule):

    def __init__(self, n_features:int):
        super().__init__()
        self.model = TempPredictionModel(n_features)
        self.criterion = nn.MSELoss

    def forward(self, x, labels=None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels.unsqueeze(dim=1))
        return loss, output

    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [63]:
model = TempPredictor(n_features=train_df.shape[1])

In [57]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
            

In [58]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint-lstm",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

logger = TensorBoardLogger("lightning_logs", name="lstm")

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5)

trainer = pl.Trainer(
    logger = logger,
    checkpoint_callback = checkpoint_callback,
    callbacks=[early_stopping_callback],
    max_epochs=20,
    gpus=1,
    progress_bar_refresh_rate=30,
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [70]:
BATCH_SIZE = 64
data_module = TempDataModule(train_sequence, test_sequence, batch_size=BATCH_SIZE)
data_module.setup()

In [73]:
data_module

1

In [72]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | TempPredictionModel | 199 K 
----------------------------------------------
199 K     Trainable params
0         Non-trainable params
199 K     Total params
0.799     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 51372) exited unexpectedly